# Assignment 2 Leaderboard
This script will package your assignment, put it in a submissions folder, generate a checksum and notify the server that you have a submission you want it to run. When it gets around to it, the server will then pull your submission and let you know if it is in the right form. Note that this is *not* a submission.

When you submit, the script will check some basic things locally before submitting your assignment. This verification will occur once more in the submission environment in an attempt to ensure you haven't forgotten anything in your submission.

## What you need to do
1. Set the folder you want to submit:
2. Set your display name

In [ ]:
import datetime
import os
submit_folder = "/home/jovyan/workshops/ass2_testing/submission"
display_name = "Anonymous"


print("[", datetime.datetime.now(), "]", "Set leaderboard folder to", submit_folder)
print("[", datetime.datetime.now(), "]", "Using", display_name, "for leaderboard display name")

3. Run the leaderboard script.

In [ ]:
import os
import subprocess
import zipfile
import submitutils
import time
from IPython.core.display import display, HTML
import html
import json
try:
    if not submit_folder:
        display(HTML('<span style="color:red">Submit folder not set</span>'))
    else:
        display(HTML('<span style="color:green">Submit folder set to: {}</span>'.format(html.escape(submit_folder))))
    if not display_name:
        display(HTML('<span style="color:red">Display name not set</span>'))
    else:
        display(HTML('<span style="color:green">Display name set to: {}</span>'.format(html.escape(display_name))))
except Exception as e:
    display(HTML('<span style="color:red">Submit folder or display name weren\'t set</span>'))
    raise

# The folder where data is persisted
BASE_FOLDER = "/home/jovyan"

# The folder where submissions should be placed, should be relative to mount point. This can be safely changed.
SUBMISSIONS_FOLDER = "submissions"

# The assignment being submitted to
ASSIGNMENT_NAME = "ass2"

# The verify script to test the submission should pass
VERIFY_SCRIPT_LOCATION = "/home/jovyan/workshops/ass2_testing/verify.sh"

# The location to store all messages recieved on the queue.
MONITOR_FILE = "/home/jovyan/assignments/submissions.log"

print("[", datetime.datetime.now(), "]", "Using submission folder", submit_folder, "for submission.")

# Write submit settings
update_submission = True
if os.path.exists(os.path.join(submit_folder, "submission.json")):
    with open(os.path.join(submit_folder, "submission.json"), "r") as f:
        existing_submission_data = json.loads(f.read())
        try:
            if existing_submission_data["display_name"] == display_name:
                update_submission = False
        except Exception as e:
            pass
if update_submission:
    output_json = {
        "display_name": display_name
    }
    with open(os.path.join(submit_folder, "submission.json"), "w+") as f:
        f.write(json.dumps(output_json, indent=4))

# Find latest file in file list.
list_of_files = submitutils.file_list(submit_folder)
list_of_files.sort(key=lambda x: x['modified_s'], reverse=True)
list_of_filenames = [(file['filename'], file['timestamp'], file['raw_filesize']) for file in list_of_files]
last_modified_file = list_of_files[0]

submission_zipfile_base = last_modified_file['timestampstring'] + '_' + ASSIGNMENT_NAME
submission_zipfile_name = submission_zipfile_base + ".zip"
print("[", datetime.datetime.now(), "]", "Storing files in zip file", submission_zipfile_name, "for submission.")

print("[", datetime.datetime.now(), "]", "Submission contents:")
print("\tFilename\tModified")
BOLD_START = "\033[1m"
END_FORMATTING = "\033[0m"
for file, modified, size in list_of_filenames:
    if modified == list_of_filenames[0][1]:
        print(BOLD_START, end="")
    print("\t" + file + "\t" + modified + "\t" + submitutils.get_friendly_size(size))
    if modified == list_of_filenames[0][1]:
        print(END_FORMATTING, end="")

base_relative_folder = SUBMISSIONS_FOLDER + "/" + ASSIGNMENT_NAME
submission_zipfile_folder = BASE_FOLDER + "/" + base_relative_folder
submission_zipfile_location = submission_zipfile_folder + "/" + submission_zipfile_name
print("[", datetime.datetime.now(), "]", "Checking if",  submission_zipfile_location, "already exists")
if os.path.exists(submission_zipfile_location):
    print("[", datetime.datetime.now(), "]", submission_zipfile_location, "already exists, delete the file if you intend to resubmit this submission.")
else:
    # Ensure the folder exists
    os.popen('mkdir -p {}'.format(submission_zipfile_folder)).read()
    with zipfile.ZipFile(submission_zipfile_location, mode="w", compression=zipfile.ZIP_LZMA) as zf:
        for file in list_of_files:
            zf.write(file['path'], arcname=file['relative_path'])
    zip_hash = submitutils.get_file_hash(submission_zipfile_location)
    print("[", datetime.datetime.now(), "]", "Submission hash:", zip_hash)
    zip_hash_location = submission_zipfile_folder + "/" + submission_zipfile_base + ".sha1"
    with open(zip_hash_location, "w") as f:
        f.write(zip_hash)
    
    # Verify submission locally
    print("")
    print("[", datetime.datetime.now(), "]", "Verifying submission locally")
    SUBMISSION_FOLDER = submit_folder
    VERIFICATION_OUTPUT_FOLDER = "/home/jovyan/assignments/output"
    os.popen("mkdir -p {}".format(VERIFICATION_OUTPUT_FOLDER)).read()
    VERIFICATION_OUTPUT_FILE = "verify_out.txt"
    USERNAME = "N/A"
    VERIFICATION_EXTRA_DATA_FOLDER = "N/A"
    TIMEOUT = 124
    os.popen("chmod 777 {}".format(VERIFY_SCRIPT_LOCATION))
    verify_command = ["/bin/bash", VERIFY_SCRIPT_LOCATION, SUBMISSION_FOLDER, VERIFICATION_OUTPUT_FOLDER, USERNAME, VERIFICATION_EXTRA_DATA_FOLDER]
    verify_success = False
    try:
        subprocess.check_call(verify_command)
        if os.path.exists(VERIFICATION_OUTPUT_FOLDER + "/" + VERIFICATION_OUTPUT_FILE):
            with open(VERIFICATION_OUTPUT_FOLDER + "/" + VERIFICATION_OUTPUT_FILE, "r") as f:
                print(f.read())
        verify_success = True
    except subprocess.CalledProcessError as e:
        
        print("[", datetime.datetime.now(), "]", "Verification failed:")
        print(e.cmd)
        if os.path.exists(VERIFICATION_OUTPUT_FOLDER + "/" + VERIFICATION_OUTPUT_FILE):
            with open(VERIFICATION_OUTPUT_FOLDER + "/" + VERIFICATION_OUTPUT_FILE, "r") as f:
                print(f.read())
        if e.returncode == TIMEOUT:
            print("Error Code:", e.returncode, "Process ran out of time")
        else:
            print("Error Code:", e.returncode, "Process ran out of time")
    if verify_success:
        # Submit to Assignment queue
        print("")
        
        # Submit mode is "submit" when user disk control is used as authorization.
        # Submit mode is "psubmit" when command signing is used as authorization.
        submit_mode = "submit"
        username = submitutils.get_username()
        print("[", datetime.datetime.now(), "]", "Putting assignment in queue")
        print("submission type: ", submit_mode)
        print("username: ", username)
        print("assignment: ", ASSIGNMENT_NAME)
        print("checksum: ", zip_hash)
        final_submit_path = base_relative_folder + "/" + submission_zipfile_name
        print("path: ", final_submit_path)
        print("submission size: ", submitutils.get_friendly_size(os.path.getsize(submission_zipfile_location)))
        if not submitutils.submit_to_queue(submit_mode="submit", username=username, assignment=ASSIGNMENT_NAME, zip_hash=zip_hash, submit_path=final_submit_path):
            raise Exception("Missing required field")
        print("[", datetime.datetime.now(), "]", "Beginning listen for queue, wait until verification completes", username)
        pass_result = submitutils.monitor_queue(queue=username, complete_stage="verify", monitor_file=MONITOR_FILE, submission_name=final_submit_path)
        if not pass_result:
            display(HTML('{} <span style="color:red">{}</span>'.format(html.escape(" ".join(["[", str(datetime.datetime.now()), "]", "Verification completed, result:"])), html.escape(str(pass_result)))))
            display(HTML('<span style="color:red">{}</span>'.format(html.escape(" ".join(["[", str(datetime.datetime.now()), "]", "Verification failed, please wait for all processing to finish and read the logs."])))))
        else:
            display(HTML('{} <span style="color:green">{}</span>'.format(html.escape(" ".join(["[", str(datetime.datetime.now()), "]", "Verification completed, result:"])), html.escape(str(pass_result)))))
            print("[", datetime.datetime.now(), "]", "Please save your submitted zip file and message log")
            submitutils.monitor_queue(queue=username, complete_stage="test", monitor_file=MONITOR_FILE, submission_name=final_submit_path)
            print("[", datetime.datetime.now(), "]", "Completed testing")
            submitutils.monitor_queue(queue=username, complete_stage="results", monitor_file=MONITOR_FILE, submission_name=final_submit_path)
            print("[", datetime.datetime.now(), "]", "Finished evaluating test results")
            submitutils.monitor_queue(queue=username, complete_stage="finalise", monitor_file=MONITOR_FILE, submission_name=final_submit_path)
            display(HTML('<span style="color:green">{}</span>'.format(html.escape(" ".join(["[", str(datetime.datetime.now()), "]", "Testing finished, results ready"])))))
            
            communication_file = submission_zipfile_folder + "/" + submission_zipfile_base + ".out"
            if os.path.exists(communication_file):
                with open(communication_file, "r") as f:
                    print(f.read())
            else:
                print("Failed to read communication file.")
        
        
    

# View Leaderboards
The leaderboards update periodically, you can see the current leaderboards either by visiting the urls:

http://comp20008-jh.eng.unimelb.edu.au:9889/leaderboard/Task1A.html

and

http://comp20008-jh.eng.unimelb.edu.au:9889/leaderboard/Task1B.html

or by running these two cells.

In [ ]:
import requests
from IPython.core.display import display, HTML
r = requests.get('http://comp20008-jh.eng.unimelb.edu.au:9889/leaderboard/Task1A.html')

display(HTML(r.text))

In [ ]:
import requests
from IPython.core.display import display, HTML
r = requests.get('http://comp20008-jh.eng.unimelb.edu.au:9889/leaderboard/Task1B.html')

display(HTML(r.text))